In [ ]:
%load_ext autoreload
%autoreload 2

# ImageBot

This notebook shows how to use the ImageBot API to generate images from text.
Underneath the hood, it uses the OpenAI API.
This bot can be combined with other bots (e.g. `SimpleBot`) to create rich content.

In [ ]:
from llamabot.bot.imagebot import ImageBot

In [ ]:
bot = ImageBot()
bot("A siamese cat.")